 # Using met.no forecasts with weather-tools



 This notebook-style script shows how to fetch met.no forecasts and combine them

 with SILO observations using the new `weather_tools` APIs. Open it directly in

 JupyterLab or VS Code with Jupytext to run the cells interactively.

 ## Prerequisites

 - Network access to `https://api.met.no` and a valid contact e-mail for the User-Agent header.

 - Set `SILO_API_KEY` (your registered SILO e-mail) or pass `api_key=` directly to `SiloAPI`.

 - Install `weather-tools` in your environment (either via `pip install weather-tools` or the local checkout).

 - Optional: enable logging (e.g. `logging.basicConfig(level=logging.INFO)`) to inspect requests.

In [1]:
import pandas as pd

from weather_tools.merge_weather_data import get_merge_summary, merge_historical_and_forecast
from weather_tools.metno_api import MetNoAPI
from weather_tools.metno_models import MetNoFormat, MetNoQuery
from weather_tools.silo_api import SiloAPI
from weather_tools.silo_models import AustralianCoordinates


## 1. Configure the met.no API client

 met.no requires a descriptive User-Agent string that includes contact details.

 Replace the placeholder below with your own application name and e-mail.

In [2]:
api = MetNoAPI(
    # user_agent="weather-tools-example/0.1 (contact: you@example.com)",
    enable_cache=True,
    log_level="DEBUG",
)


## 2. Build a forecast query for your location

 The `AustralianCoordinates` model validates that the latitude and longitude sit

 within the range supported by SILO (GDA94 datum). Adjust the coordinates to

 target your site of interest.

In [3]:
coordinates = AustralianCoordinates(latitude=-33.86, longitude=151.21)  # Sydney, NSW
query = MetNoQuery(coordinates=coordinates, format=MetNoFormat.COMPACT)

response = api.query_forecast(query)
metadata = response.get_meta()

metadata


[10/23/25 22:26:00] DEBUG    🌐 Constructed URL:                                                                   
                             https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=-33.86&lon=151.21

                    DEBUG    📋 User-Agent: weather-tools/0.0.0 (Python 3.12)

                    DEBUG    Making request (attempt 1/3):                                                         
                             https://api.met.no/weatherapi/locationforecast/2.0/compact

[10/23/25 22:26:01] DEBUG    Cached response for: 4a5961d27789c123038238e4def09a11

                    INFO     Request successful on attempt 1

{'updated_at': '2025-10-23T13:16:43Z',
 'units': {'air_pressure_at_sea_level': 'hPa',
  'air_temperature': 'celsius',
  'cloud_area_fraction': '%',
  'precipitation_amount': 'mm',
  'relative_humidity': '%',
  'wind_from_direction': 'degrees',
  'wind_speed': 'm/s'}}

 ## 3. Convert forecasts to daily and hourly tables

 `MetNoAPI.to_dataframe` can aggregate the hourly GeoJSON payload to daily summaries

 or return hourly data. Use the `frequency` parameter to control aggregation:

 - `frequency='daily'` (default) - Daily aggregates

 - `frequency='hourly'` - Raw hourly data

 - `frequency='weekly'` or `frequency='monthly'` - Other time periods

In [4]:
daily_forecasts = api.to_dataframe(response, frequency='daily')
hourly_forecasts = api.to_dataframe(response, frequency='hourly')

daily_forecasts.head()

,date,min_temperature,max_temperature,total_precipitation,avg_wind_speed,max_wind_speed,avg_relative_humidity,avg_pressure,avg_cloud_fraction,dominant_weather_symbol
0,2025-10-23,16.6,21.2,0.0,3.700000,5.0,68.490000,1013.390000,35.310000,partlycloudy_night
1,2025-10-24,15.0,23.8,0.3,5.612500,11.2,70.179167,1015.633333,66.466667,lightrain
2,2025-10-25,15.6,21.9,0.0,5.704167,9.3,74.204167,1011.820833,54.725000,cloudy
3,2025-10-26,18.7,24.6,1.7,4.433333,7.8,52.333333,1003.044444,62.233333,rain
4,2025-10-27,14.8,24.9,1.7,7.650000,10.8,54.300000,1010.400000,66.025000,rainshowers_day


In [5]:
hourly_forecasts.head()


,time,air_pressure_at_sea_level,air_temperature,cloud_area_fraction,relative_humidity,wind_from_direction,wind_speed,precipitation_amount,symbol_code
0,2025-10-23 14:00:00,1014.4,18.1,43.0,65.9,22.4,5.0,0.0,partlycloudy_night
1,2025-10-23 15:00:00,1013.7,17.4,20.3,71.9,355.7,4.3,0.0,fair_night
2,2025-10-23 16:00:00,1013.1,17.0,30.5,73.7,293.5,3.0,0.0,fair_night
3,2025-10-23 17:00:00,1012.9,16.6,100.0,74.2,268.5,2.8,0.0,cloudy
4,2025-10-23 18:00:00,1012.7,16.8,97.7,72.6,265.4,3.5,0.0,cloudy


 ### Quick helper: `get_daily_forecast`

 `get_daily_forecast` returns a pandas DataFrame directly with daily summaries,

 making it perfect for quick analysis and merging with SILO data.

In [6]:
daily_df = api.get_daily_forecast(
    latitude=coordinates.latitude,
    longitude=coordinates.longitude,
    days=5,
)

daily_df  # Already a DataFrame!

                    DEBUG    🌐 Constructed URL:                                                                   
                             https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=-33.86&lon=151.21

                    DEBUG    📋 User-Agent: weather-tools/0.0.0 (Python 3.12)

                    DEBUG    Cache hit for request: 4a5961d27789c123038238e4def09a11

,date,min_temperature,max_temperature,total_precipitation,avg_wind_speed,max_wind_speed,avg_relative_humidity,avg_pressure,avg_cloud_fraction,dominant_weather_symbol
0,2025-10-23,16.6,21.2,0.0,3.700000,5.0,68.490000,1013.390000,35.310000,partlycloudy_night
1,2025-10-24,15.0,23.8,0.3,5.612500,11.2,70.179167,1015.633333,66.466667,lightrain
2,2025-10-25,15.6,21.9,0.0,5.704167,9.3,74.204167,1011.820833,54.725000,cloudy
3,2025-10-26,18.7,24.6,1.7,4.433333,7.8,52.333333,1003.044444,62.233333,rain
4,2025-10-27,14.8,24.9,1.7,7.650000,10.8,54.300000,1010.400000,66.025000,rainshowers_day


## 4. Merge met.no forecasts with SILO history

 Pull the last five days from the SILO DataDrill API for the same coordinates,

 then merge that history with the met.no forecast. The helper automatically

 converts column names, fills optional variables (when enabled), and annotates

 the data source for each record.

In [7]:
daily_forecasts.date

0    2025-10-23
1    2025-10-24
2    2025-10-25
3    2025-10-26
4    2025-10-27
5    2025-10-28
6    2025-10-29
7    2025-10-30
8    2025-10-31
9    2025-11-01
10   2025-11-02
Name: date, dtype: datetime64[ns]

In [8]:
first_forecast_date = pd.to_datetime(daily_forecasts["date"]).min()
history_end = first_forecast_date - pd.Timedelta(days=1)
history_start = history_end - pd.Timedelta(days=4)

silo_api = SiloAPI(log_level="INFO")
raw_silo = silo_api.get_gridded_data(
    latitude=coordinates.latitude,
    longitude=coordinates.longitude,
    start_date=history_start.strftime("%Y%m%d"),
    end_date=history_end.strftime("%Y%m%d"),
    variables=["rainfall", "max_temp", "min_temp"],
)

silo_history = raw_silo.rename(columns={"YYYY-MM-DD": "date"}).copy()

silo_history["date"] = pd.to_datetime(silo_history["date"])

required_cols = ["date", "min_temp", "max_temp", "daily_rain"]
missing = [col for col in required_cols if col not in silo_history.columns]
if missing:
    raise ValueError(f"Missing required columns from SILO response: {missing}")

silo_data = silo_history[required_cols].sort_values("date").reset_index(drop=True)

merged = merge_historical_and_forecast(
    silo_data=silo_data,
    metno_data=daily_forecasts,
    fill_missing=True,
    overlap_strategy="prefer_silo",
)

merged


[10/23/25 22:26:07] INFO     Request successful on attempt 1

                    INFO     Auto-detected transition date: 2025-10-23 00:00:00

/Users/hfsi/Developer/weather_tools/src/weather_tools/merge_weather_data.py:149: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([silo_df, metno_df], ignore_index=True)


                    INFO     Merged 5 SILO records with 11 met.no records. Total: 16 records

,min_temp,data_source,date,avg_wind_speed,max_wind_speed,vp,forecast_generated_at,avg_cloud_fraction,dominant_weather_symbol,is_forecast,mslp,daily_rain,max_temp
0,15.7,silo,2025-10-18,NaN,NaN,NaN,NaT,NaN,NaN,False,NaN,0.7,22.0
1,15.5,silo,2025-10-19,NaN,NaN,NaN,NaT,NaN,NaN,False,NaN,0.0,25.5
2,17.0,silo,2025-10-20,NaN,NaN,NaN,NaT,NaN,NaN,False,NaN,0.0,36.0
3,17.7,silo,2025-10-21,NaN,NaN,NaN,NaT,NaN,NaN,False,NaN,0.0,23.7
4,18.7,silo,2025-10-22,NaN,NaN,NaN,NaT,NaN,NaN,False,NaN,0.0,22.8
5,16.6,metno,2025-10-23,3.700000,5.0,68.490000,2025-10-23 14:26:07.396766+00:00,35.310000,partlycloudy_night,True,1013.390000,0.0,21.2
6,15.0,metno,2025-10-24,5.612500,11.2,70.179167,2025-10-23 14:26:07.396766+00:00,66.466667,lightrain,True,1015.633333,0.3,23.8
7,15.6,metno,2025-10-25,5.704167,9.3,74.204167,2025-10-23 14:26:07.396766+00:00,54.725000,cloudy,True,1011.820833,0.0,21.9
8,18.7,metno,2025-10-26,4.433333,7.8,52.333333,2025-10-23 14:26:07.396766+00:00,62.233333,rain,True,1003.044444,1.7,24.6
9,14.8,metno,2025-10-27,7.650000,10.8,54.300000,2025-10-23 14:26:07.396766+00:00,66.025000,rainshowers_day,True,1010.400000,1.7,24.9


In [9]:
merge_summary = get_merge_summary(merged)
merge_summary


{'total_records': 16,
 'silo_records': np.int64(5),
 'metno_records': np.int64(11),
 'date_range': {'start': Timestamp('2025-10-18 00:00:00'),
  'end': Timestamp('2025-11-02 00:00:00'),
  'days': 16},
 'silo_period': {'start': Timestamp('2025-10-18 00:00:00'),
  'end': Timestamp('2025-10-22 00:00:00')},
 'metno_period': {'start': Timestamp('2025-10-23 00:00:00'),
  'end': Timestamp('2025-11-02 00:00:00')},
 'transition_date': Timestamp('2025-10-23 00:00:00')}

## 4b. Merge Forecast with SILO PatchedPoint Data

In [10]:

first_forecast_date = pd.to_datetime(daily_forecasts["date"]).min()
history_end = first_forecast_date #- pd.Timedelta(days=1) # Include day before forecast to check overlap handling
history_start = history_end - pd.Timedelta(days=8)

silo_api = SiloAPI(log_level="INFO")
raw_silo = silo_api.search_stations("Northam")

# {'station_code': 10111,
#  'name': 'NORTHAM',
#  'latitude': -31.651,
#  'longitude': 116.659,
#  'state': 'WA',
#  'elevation': 170.0}

silo_station_data = silo_api.get_station_data(station_code="10111",
						  start_date=history_start.strftime("%Y%m%d"),
						end_date=history_end.strftime("%Y%m%d"),
						variables=["rainfall", "max_temp", "min_temp", "evaporation"],
		)

silo_station_data = silo_station_data.rename(columns={"YYYY-MM-DD": "date"}).copy()


merge_historical_and_forecast(
    silo_data=silo_station_data,
    metno_data=daily_forecasts,
    fill_missing=True,
    overlap_strategy="prefer_silo",
)



[10/23/25 22:26:10] INFO     Request successful on attempt 1

[10/23/25 22:26:11] INFO     Request successful on attempt 1

                    INFO     Auto-detected transition date: 2025-10-23 00:00:00

/Users/hfsi/Developer/weather_tools/src/weather_tools/merge_weather_data.py:149: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([silo_df, metno_df], ignore_index=True)


                    INFO     Merged 8 SILO records with 11 met.no records. Total: 19 records

,min_temp_source,min_temp,data_source,date,avg_wind_speed,max_wind_speed,vp,evap_pan,forecast_generated_at,avg_cloud_fraction,evap_pan_source,dominant_weather_symbol,station,is_forecast,daily_rain_source,mslp,daily_rain,max_temp,max_temp_source
0,0.0,10.6,silo,2025-10-15,NaN,NaN,NaN,4.6,NaT,NaN,25.0,NaN,10111.0,False,0.0,NaN,0.0,24.7,0.0
1,0.0,8.0,silo,2025-10-16,NaN,NaN,NaN,6.7,NaT,NaN,25.0,NaN,10111.0,False,0.0,NaN,0.0,27.7,0.0
2,0.0,9.1,silo,2025-10-17,NaN,NaN,NaN,6.8,NaT,NaN,25.0,NaN,10111.0,False,0.0,NaN,0.0,34.0,0.0
3,0.0,15.9,silo,2025-10-18,NaN,NaN,NaN,6.9,NaT,NaN,25.0,NaN,10111.0,False,0.0,NaN,0.0,24.4,0.0
4,0.0,7.5,silo,2025-10-19,NaN,NaN,NaN,4.9,NaT,NaN,25.0,NaN,10111.0,False,0.0,NaN,0.0,22.9,0.0
5,0.0,6.0,silo,2025-10-20,NaN,NaN,NaN,3.2,NaT,NaN,25.0,NaN,10111.0,False,0.0,NaN,0.0,21.4,0.0
6,0.0,9.0,silo,2025-10-21,NaN,NaN,NaN,4.5,NaT,NaN,25.0,NaN,10111.0,False,0.0,NaN,19.8,22.5,0.0
7,0.0,8.7,silo,2025-10-22,NaN,NaN,NaN,0.0,NaT,NaN,0.0,NaN,10111.0,False,0.0,NaN,0.0,25.5,75.0
8,NaN,16.6,metno,2025-10-23,3.700000,5.0,68.490000,NaN,2025-10-23 14:26:11.175867+00:00,35.310000,NaN,partlycloudy_night,NaN,True,NaN,1013.390000,0.0,21.2,NaN
9,NaN,15.0,metno,2025-10-24,5.612500,11.2,70.179167,NaN,2025-10-23 14:26:11.175867+00:00,66.466667,NaN,lightrain,NaN,True,NaN,1015.633333,0.3,23.8,NaN


In [11]:
daily_forecasts.columns

Index(['date', 'min_temperature', 'max_temperature', 'total_precipitation',
       'avg_wind_speed', 'max_wind_speed', 'avg_relative_humidity',
       'avg_pressure', 'avg_cloud_fraction', 'dominant_weather_symbol'],
      dtype='object')